In [1]:
import findspark

findspark.init()

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
import json
import pickle

sc = SparkContext()
ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)

socket_stream = ssc.socketTextStream("localhost", 9009)

lines = socket_stream.window(20,20)

lines.map( lambda text: text.split( "~@" ) ).foreachRDD(lambda rdd: rdd.toDF().registerTempTable("tweets"))

In [2]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
# Only works for Jupyter Notebooks!
%matplotlib inline

ssc.start()

In [5]:
count = 0
while count < 100:
    
    time.sleep( 2 )
    
    top_10_source = sqlContext.sql('select * from tweets' )
    top_10_df = top_10_source.toPandas()
    print(top_10_df)

                     _1                                                 _2  \
0  1195131908386164736    RT @PDI_Araucania: Tras un trabajo investigat...   
1  1195131908377600000    RT @FunctionalCoach: Having “reasons” isn’t a...   
2  1195131908352380928    RT @RonniSalt: And tweepers - here's the answ...   
3  1195131908419674114    RT @Erikaatierra: @_Luky__ @luisyepesb @Caden...   
4  1195131908356575233    RT @NanyPardo: Si el @mindefensa lo dice...Al...   
5  1195131908302221313    RT @_KaylD0m: Alguém me explica isso??? Nível...   
6  1195131908218327040                                         Queen ♥️🎧🎶    
7  1195131908235124736    RT @eliefares: Yo @scottborchetta &amp; @scoo...   

                      _3      _4     _5                    _6  
0             189742061    None    1.0       Criminal Justice  
1    739236690087186434    None    0.0       Science & Health  
2    778893466939994112    None    0.0    Policy and Politics  
3             194217605    None    1.0   

KeyboardInterrupt: 